## Evaluation Script Using the Experiment's Folder

#### This script uses the not the files contained within the experiment's directory to obtain results. While it is slightly complex at the moment, it ensures backward compatibility in case the content of the python files change. This uses the scripts used for generating the experiments.

In [ ]:
import os 
import sys
from pathlib import Path
import yaml

PATH_TO_HOME = '/home/' + os.getenv('USER') + '/voxattack'
sys.path.append(PATH_TO_HOME)

os.chdir(f'{PATH_TO_HOME}')

## Experiment Params

In [ ]:
# Define tour experiment
path_to_exp = ... # 'path/to/your/experiment'  # Change this to your experiment path

paths_to_metadata = f"{path_to_exp}/metadata"
paths_to_configs = f"{paths_to_metadata}/configs"
paths_to_src = f"{path_to_exp}/src"
os.chdir(f'{paths_to_configs}')

ckpt_path = [ckpt for ckpt in os.listdir(f"{path_to_exp}/checkpoints") if ckpt != 'last.ckpt']
assert len(ckpt_path) == 1, 'Expected a single model'
ckpt_path = f"{path_to_exp}/checkpoints/{ckpt_path[0]}"

In [ ]:
rel_path_eval = "../src/eval.py"
module = f"vpc.yaml"
datamodule = f"datasets/vpc.yaml"
experiment = f"vpc/voxattack.yaml"

## Load Overrides and Modify

In [ ]:
overrides_path = Path("../../.hydra/overrides.yaml")

with open(overrides_path, 'r') as file:
    overrides_data_list = yaml.safe_load(file)

overrides_data = " ".join([f"'{item}'" if '{' in item and '}' in item else (item if 'logger' not in item else '') for item in overrides_data_list])
overrides_data

### Hot Fixes

In [ ]:
# 1. Update eval.py: test --> validate: metadata/src/eval.py

# 2. Commend defaults in experiment config: /metadata/configs/experiment/vpc/EXP.yaml

# 3. [for eval dev] Cohort Embeddings: train_data is None in metadata/src/datamodules/vpc_datamodule.py
# if stage == 'fit' or stage is None: --> if True:

# 4. [OPTIONAL - for eval] update: metadata/src/modules/multimodal_vpc.py
# override/write embedding type + set cohort_path = None


#=============
# clusters exps
overrides_data = overrides_data.replace("trainer.num_sanity_val_steps=0","")
overrides_data = overrides_data.replace("name=...", "")
overrides_data = overrides_data.replace("experiment=...", "")
overrides_data = overrides_data.replace("paths.data_dir=...", "")
overrides_data = overrides_data.replace("datamodule.loaders.valid.batch_size=...", "")
overrides_data = overrides_data.replace("paths.log_dir=...", f'paths.log_dir={path_to_exp}')
overrides_data = overrides_data.replace("ckpt_path=.../checkpoints/last.ckpt", "")
overrides_data

In [ ]:
env_file_path = f"{paths_to_metadata}/.env"

if not os.path.exists(env_file_path):
    with open(env_file_path, 'w') as env_file:
        env_file.write(f"PROJECT_ROOT={paths_to_metadata}/voxattack\n")
        # Add other environment variables as needed

In [ ]:
!python $rel_path_eval +experiment=$experiment $overrides_data ckpt_path=$ckpt_path paths.data_dir=$PATH_TO_HOME/data \
module.scores_norm.embeds_metric_params.cohort_per_model=40000 module.scores_norm.scores_norm_params.topk=1000 module.scores_norm.scores_norm_params.min_cohort_size=10